In [1]:
from nltk.corpus import stopwords
import nltk
from nltk import word_tokenize, FreqDist
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from gensim.models import Word2Vec
import pandas as pd

In [2]:
text = pd.read_csv('combined_text_data.csv')
stock_orignial = pd.read_csv('stock_data.csv')
stock = pd.read_csv('interpolated_stock_data.csv')

In [3]:
# stock = stock.iloc[::-1]

In [4]:
stock.rename(columns={'Unnamed: 0':'Date'}, inplace=True)

In [5]:
stock.shape

(1776, 2)

In [6]:
stock.head()

,Date,signal
0,2012-11-19,up
1,2012-11-20,stay
2,2012-11-21,stay
3,2012-11-22,down
4,2012-11-23,down


In [7]:
text.shape

(3205, 3859)

In [8]:
text.columns

Index(['Time', 'retweet_from', 'tweet_length', 'encoded_sentiment',
       'sentiment', 'polarity', 'tweet', 'ability', 'able', 'aboard',
       ...
       'you', 'your', 'yours', 'yourself', 'yr', 'yup', 'zero', 'zip', 'zone',
       'zoo'],
      dtype='object', length=3859)

In [9]:
text.head()

,Time,retweet_from,tweet_length,encoded_sentiment,sentiment,polarity,tweet,ability,able,aboard,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,2017-09-29,NaN,105,1,positive,0.366667,"Assuming max acceleration of 2 to 3 g's, but i...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-09-29,SpaceX,113,1,positive,0.200000,BFR is capable of transporting satellites to o...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-09-29,NaN,6,0,neutral,0.000000,Yup :),0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2017-09-29,NaN,7,0,neutral,0.000000,Part 2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-09-29,NaN,96,1,positive,0.650000,Fly to most places on Earth in under 30 mins a...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
stock.set_index('Date', inplace=True)

In [11]:
idx = pd.date_range('2012-11-19', '2017-09-29')

# s = pd.Series({'09-02-2013': 2,
#                '09-03-2013': 10,
#                '09-06-2013': 5,
#                '09-07-2013': 1})

# s.index = pd.DatetimeIndex(s.index)
# s = s.reindex(idx, fill_value='NaN')
# print(s)

In [12]:
stock.index = pd.DatetimeIndex(stock.index)

In [13]:
stock = stock.reindex(idx, fill_value='NaN')

In [22]:
stock.index.name = 'Date'

---

In [15]:
text.rename(columns={'Time':'Date'}, inplace=True)

In [25]:
text = text.set_index('Date')

In [28]:
text.index = pd.DatetimeIndex(text.index)

In [29]:
stock.head()

,signal
Date,
2012-11-19,up
2012-11-20,stay
2012-11-21,stay
2012-11-22,down
2012-11-23,down


In [30]:
take_2 = pd.merge(text, stock, on='Date', how='left')

In [31]:
take_2.shape

(3205, 3859)

In [32]:
take_2.head(10)

,retweet_from,tweet_length,encoded_sentiment,sentiment,polarity,tweet,ability,able,aboard,abort,...,your,yours,yourself,yr,yup,zero,zip,zone,zoo,signal_y
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-29,NaN,105,1,positive,0.366667,"Assuming max acceleration of 2 to 3 g's, but i...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
2017-09-29,SpaceX,113,1,positive,0.200000,BFR is capable of transporting satellites to o...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
2017-09-29,NaN,6,0,neutral,0.000000,Yup :),0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,stay
2017-09-29,NaN,7,0,neutral,0.000000,Part 2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
2017-09-29,NaN,96,1,positive,0.650000,Fly to most places on Earth in under 30 mins a...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
2017-09-29,SpaceX,80,1,positive,0.125000,"Supporting the creation of a permanent, self-s...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
2017-09-29,NaN,57,-1,negative,-0.166667,BFR will take you anywhere on Earth in less th...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
2017-09-29,NaN,84,0,neutral,0.000000,Mars City Opposite of Earth. Dawn and dusk sky...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay
2017-09-29,NaN,16,-1,negative,-0.800000,Moon Base Alpha,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stay


In [33]:
take_2.signal_y.value_counts()

stay    1311
up       993
down     900
Name: signal_y, dtype: int64

---

#### PCAAAAAAAA

In [34]:
from sklearn.decomposition import PCA

In [64]:
pca = PCA(n_components=800)

In [65]:
tfidf_df = take_2.iloc[:, 7:-1]

In [66]:
tfidf_df.head()

,able,aboard,abort,aborted,about,above,absolutely,absorb,absorbed,absorption,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-09-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-09-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2017-09-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-09-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
pca.fit(tfidf_df)

PCA(copy=True, iterated_power='auto', n_components=800, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [68]:
x_pca = pca.transform(tfidf_df)
x_pca

array([[-0.03175036, -0.02885961, -0.03027349, ..., -0.00589298,
         0.02021594, -0.02442969],
       [-0.05933081, -0.09982727, -0.15475698, ...,  0.01911102,
        -0.01190816,  0.00038602],
       [-0.00233025,  0.0339694 ,  0.06957776, ..., -0.00258265,
        -0.00109961,  0.00062738],
       ...,
       [-0.02647321, -0.01465776,  0.02605385, ...,  0.00165889,
        -0.01636098, -0.00567281],
       [-0.02481626, -0.00748991,  0.0050595 , ...,  0.00048641,
         0.01166383, -0.02173949],
       [-0.02823683, -0.01386954,  0.00580379, ...,  0.02967191,
        -0.00637943,  0.02213245]])

In [69]:
sum(pca.explained_variance_ratio_)

0.7904432190368588

In [70]:
pca_df = pd.DataFrame(x_pca)

In [71]:
pca_df.head()

,0,1,2,3,4,5,6,7,8,9,...,790,791,792,793,794,795,796,797,798,799
0,-0.031750,-0.028860,-0.030273,-0.018769,-0.060676,0.057752,-0.124476,0.116654,0.059117,-0.029701,...,-0.014750,0.006987,-0.018947,0.003795,0.021159,0.007026,-0.077665,-0.005893,0.020216,-0.024430
1,-0.059331,-0.099827,-0.154757,-0.040344,0.072303,0.209183,-0.080122,0.077753,-0.073507,-0.089249,...,0.003511,-0.017758,-0.007694,0.003239,0.015531,0.019036,0.019721,0.019111,-0.011908,0.000386
2,-0.002330,0.033969,0.069578,-0.014138,-0.037713,-0.071401,0.007006,-0.015473,-0.033879,0.008302,...,0.000253,0.001255,-0.001372,-0.001781,0.001782,0.001916,-0.002330,-0.002583,-0.001100,0.000627
3,-0.005050,0.024121,0.043358,-0.001800,-0.016294,-0.051990,-0.001818,-0.001254,-0.008862,-0.027695,...,-0.001174,-0.005936,-0.005910,0.017547,-0.001422,-0.002220,0.004902,0.000039,0.011071,0.011174
4,-0.031167,-0.027728,-0.012323,-0.039920,-0.070452,0.019524,-0.127557,-0.070038,0.074605,0.034145,...,0.003524,0.002561,-0.003482,0.002877,-0.018114,-0.014788,-0.011140,0.006785,0.002626,0.015714


In [73]:
pca_df.to_csv('r/Users/sproul/Desktop/ds-projects/project_mod4_AAR/pca_updated_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'r/Users/sproul/Desktop/ds-projects/project_mod4_AAR/pca_updated_data.csv'

In [ ]:
# pd.get_dummies(stock)

In [ ]:
labels = []

for i in range(1, 39):
    labels.append('pc_' + str(i))

In [ ]:
# labels

In [ ]:
pca_df.columns = labels

In [ ]:
df_with_tfidf_pca = pd.concat([take_2.iloc[:, :7], pca_df], axis=1)

In [ ]:
df_with_tfidf_pca

In [ ]:
df_with_tfidf_pca['retweet_from'] = df_with_tfidf_pca['retweet_from'].fillna('not')

In [ ]:
df_with_tfidf_pca['retweet_from'] = le.fit_transform(df_with_tfidf_pca['retweet_from'])

In [ ]:
df_with_tfidf_pca.drop(columns = ['sentiment', 'Date'], inplace= True)

In [ ]:
df_with_tfidf_pca.info()

In [ ]:
# take_23 = pd.merge(stock, text, on ='Date')   
# 2012-11-19   2017-09-29

In [ ]:
# this is without weekends
# take_23.shape

In [ ]:
# take_23

In [ ]:
# take_23['retweet_from'] = take_23['retweet_from'].fillna('not')

In [ ]:
# take_23.shape

In [ ]:
# take_23.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
take_23['retweet_from'] = le.fit_transform(take_23['retweet_from'])

In [ ]:
take_23.drop(columns = ['sentiment'], inplace= True)

In [ ]:
#Same but with no date
take_23.drop(columns = ['Date'], inplace= True)

#### fixing the tweet column to match cleaned tweets

In [ ]:
tweets = df_with_tfidf_pca['tweet'].tolist()

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
english = set(nltk.corpus.words.words()+ ['yup'])

In [ ]:
def lemmadata(doc):
        pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
        raw_tokens = nltk.regexp_tokenize(doc, pattern)
        tokens = [i.lower() for i in raw_tokens]
        lemmatized = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
        words = list(filter(lambda w: w in english, lemmatized))
        return words

In [ ]:
lemma = [lemmadata(post) for post in tweets]

In [ ]:
process_lems = list(map(lemmadata, df_with_tfidf_pca['tweet']))

In [ ]:
# process_lems[:3]

In [ ]:
clean_tweets_lem = []

for i in range(0, len(process_lems)):
    clean = ' '.join(process_lems[i])
    clean_tweets_lem.append(clean)

In [ ]:
# df['B'] = df1['E'].values

In [ ]:
clean_tweet_df = pd.DataFrame(clean_tweets_lem)

In [ ]:
# clean_tweet_df.head()

In [ ]:
clean_tweet_df[0][0]

In [ ]:
df_with_tfidf_pca['tweet'] = clean_tweet_df[0].values

In [ ]:
# we added signal to the dataframe
df_with_tfidf_pca = pd.concat([df_with_tfidf_pca, take_2.iloc[:,-1]], axis=1)

In [197]:
df_with_tfidf_pca.head()

,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,pc_1,pc_2,pc_3,pc_4,pc_5,...,pc_30,pc_31,pc_32,pc_33,pc_34,pc_35,pc_36,pc_37,pc_38,signal_y
0,171,105,1,0.366667,assuming acceleration of to but in a comfortab...,-0.031750,-0.028850,-0.030290,-0.018736,-0.060675,...,0.017885,0.045822,-0.006862,-0.012954,-0.005468,-0.100572,-0.029374,0.041363,0.035932,stay
1,77,113,1,0.200000,is capable of transporting satellite to orbit ...,-0.059331,-0.099826,-0.154772,-0.040355,0.072285,...,-0.011667,0.003882,0.018002,-0.019813,-0.035809,-0.034079,-0.014687,-0.008986,-0.019845,stay
2,171,6,0,0.000000,yup,-0.002330,0.033983,0.069531,-0.014177,-0.037774,...,0.015722,-0.025726,0.065735,0.041686,-0.021751,-0.014138,-0.037047,-0.050031,-0.084722,stay
3,171,7,0,0.000000,part,-0.005050,0.024117,0.043362,-0.001804,-0.016279,...,-0.009052,-0.008601,0.001285,-0.007619,-0.005289,-0.025700,-0.008989,-0.011967,-0.004665,stay
4,171,96,1,0.650000,fly to most place on earth in under min and an...,-0.031167,-0.027725,-0.012331,-0.039909,-0.070462,...,0.020042,0.030526,-0.039462,0.088186,-0.012277,0.033849,0.040576,0.051975,-0.031236,stay


In [199]:
df_with_tfidf_pca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3205 entries, 0 to 3204
Data columns (total 44 columns):
retweet_from         3205 non-null int64
tweet_length         3205 non-null int64
encoded_sentiment    3205 non-null int64
polarity             3205 non-null float64
tweet                3205 non-null object
pc_1                 3205 non-null float64
pc_2                 3205 non-null float64
pc_3                 3205 non-null float64
pc_4                 3205 non-null float64
pc_5                 3205 non-null float64
pc_6                 3205 non-null float64
pc_7                 3205 non-null float64
pc_8                 3205 non-null float64
pc_9                 3205 non-null float64
pc_10                3205 non-null float64
pc_11                3205 non-null float64
pc_12                3205 non-null float64
pc_13                3205 non-null float64
pc_14                3205 non-null float64
pc_15                3205 non-null float64
pc_16                3205 non-null float

In [ ]:
# take_23.to_csv(r'/Users/sproul/Desktop/ds-projects/project_mod4_AAR/CLEAN_EDIT_model_data.csv')

In [200]:
df_with_tfidf_pca.to_csv(r'/Users/sproul/Desktop/ds-projects/project_mod4_AAR/updated_merged_data.csv')